In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
#from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

!conda install -c conda-forge xlrd --yes

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    lib

In [2]:
#df=pd.read_excel('crimeBerlin.xlsx')
xls = pd.ExcelFile('crimeBerlin.xlsx')
df1 = pd.read_excel(xls, 'Titel')
df2 = pd.read_excel(xls, 'Inhaltsverzeichnis')
df2.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,Kriminalitätsatlas Berlin,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,Tabellen,NaN,NaN
4,NaN,NaN,NaN


In [49]:
df3 = pd.read_excel(xls, 'Fallzahlen_2018') # up to 2018
df3.drop(df3.tail(2).index,inplace=True) # drop last 2 rows
df3.drop(df3.head(4).index,inplace=True) # drop first 4 rows
newcols=['Borough','Neighborhood', 'Total records', 'Robbery', 'Street robbery', \
      'Total physical injury', 'Serious bodily harm', 'Coercion', 'Total theft', \
      'Bicycle theft', 'Burglary', 'Property damage', 'Drug offenses']
df3 = df3.drop(df3.columns[[9,10,13,14,16,18]], axis=1)
df3.columns=newcols
df3.reset_index(drop=True, inplace=True)
df3.head()

,Borough,Neighborhood,Total records,Robbery,Street robbery,Total physical injury,Serious bodily harm,Coercion,Total theft,Bicycle theft,Burglary,Property damage,Drug offenses
0,010000,Mitte,85227,733,449,7450,1854,2068,37802,4173,775,5528,3792
1,010111,Tiergarten Süd,5171,70,58,416,125,103,2540,310,39,284,273
2,010112,Regierungsviertel,9146,34,16,519,110,114,3883,372,35,407,133
3,010113,Alexanderplatz,19275,130,80,1531,366,309,10144,880,133,1036,971
4,010114,Brunnenstraße Süd,4207,26,14,286,60,69,1817,314,56,442,69


In [50]:
lorkeys=['010000','020000','030000','040000','050000','060000','070000','080000','090000','100000','110000','120000']
myIndex=[]
for tkey in lorkeys:
    df3ind=df3.index[df3['Borough'] == tkey].to_list()
    myIndex.append(df3ind[0])
myIndex = [ int(x) for x in myIndex ]
print(myIndex)
myBorough=['Mitte','Friedrichshain-Kreuzberg','Pankow', 'Charlottenburg-Wilmersdorf', 'Spandau', 'Steglitz-Zehlendorf', \
          'Tempelhof-Schöneberg', 'Neukölln', 'Treptow-Köpenick', 'Marzahn-Hellersdorf', 'Lichtenberg','Reinickendorf']
myb=df3['Borough'].astype(int).to_list()
start=10000
k=0
myBor=[]
for i in myb:
    if i < start+10000:
        myBor.append(myBorough[k])
    else:
        k=k+1
        myBor.append(myBorough[k])
        start=start+10000
        
df3['Borough']=myBor
df3.drop(myIndex,axis=0, inplace=True)
df3.head()

[0, 12, 22, 40, 59, 70, 80, 89, 101, 123, 134, 149]


,Borough,Neighborhood,Total records,Robbery,Street robbery,Total physical injury,Serious bodily harm,Coercion,Total theft,Bicycle theft,Burglary,Property damage,Drug offenses
1,Mitte,Tiergarten Süd,5171,70,58,416,125,103,2540,310,39,284,273
2,Mitte,Regierungsviertel,9146,34,16,519,110,114,3883,372,35,407,133
3,Mitte,Alexanderplatz,19275,130,80,1531,366,309,10144,880,133,1036,971
4,Mitte,Brunnenstraße Süd,4207,26,14,286,60,69,1817,314,56,442,69
5,Mitte,Moabit West,7257,71,38,653,164,215,2470,447,83,496,435


In [51]:
df3.to_csv("berlincrime2018.csv")

In [55]:
!conda install -c conda-forge lxml --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.4.1                 |   py36h7ec2d77_0         1.6 MB  conda-forge

The following packages will be UPDATED:

    lxml: 4.2.5-py37hefd8a0e_0 --> 4.4.1-py36h7ec2d77_0 conda-forge


lxml-4.4.1           | 1.6 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [83]:
!conda install -c conda-forge beautifulsoup4 --yes
#!pip install 

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    soupsieve-1.9.4            |           py36_0          58 KB  conda-forge
    beautifulsoup4-4.8.1       |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         207 KB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.4-py36_0 conda-forge

The following packages will be UPDATED:

    beautifulsoup4: 4.6.3-py37_0             --> 4.8.1-py36_0 conda-forge


soupsieve-1.9.4      

In [219]:
from bs4 import BeautifulSoup
import re
content = requests.get(url)
soup = BeautifulSoup(content.text, 'html.parser')

myList=[]

for br in soup.find_all("br"):
        br.replace_with("\n\n")
        
rows = soup.find_all('tr') # Extract and return first occurrence of tr
for row in rows:
    text = str(row.get_text())
    b=text.split("\n\n")
    myList.append(b)

In [229]:
df=pd.DataFrame(myList)
df.columns=['T','ID','School','Borough','Grade']
df.drop(df.head(1).index,inplace=True)
df.drop(columns=['T','ID'],inplace=True)
df = df.replace('\n','', regex=True)
df.head()

,School,Borough,Grade
1,Heinrich-Hertz-Gymnasium,Friedrichshain,"1,76"
2,Otto-Nagel-Gymnasium,Biesdorf,"1,81"
3,Rosa-Luxemburg-Gymnasium,Pankow,"1,82"
4,Französisches Gymnasium,Tiergarten,"1,85"
5,Arndt-Gymnasium Dahlem,Dahlem,"1,85"


In [230]:
df.to_csv("berlinschool2019.csv")